<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#PS" data-toc-modified-id="PS-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PS</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Jak-wygląda-struktura-interpertacji?" data-toc-modified-id="Jak-wygląda-struktura-interpertacji?-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Jak wygląda struktura interpertacji?</a></span></li><li><span><a href="#Podział-na-tokeny" data-toc-modified-id="Podział-na-tokeny-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Podział na tokeny</a></span></li></ul></li></ul></li></ul></div>

## PS

#### Jak wygląda struktura interpertacji? 

Intepretacje ściągnąłem za pomocą webscrapera z [bazy Ministerstwa Finansów](http://sip.mf.gov.pl), natomiast klasyfikacja była dość bardziej skomplikowana.

W ściągniętej interpretacji w pewny sposób można wyodrębnić następujące informacje:

1. dzień, miesiąc i rok wydania interpretacji,

2. tekst interpretacji.

3. autor

4. sygnatura

W systemie SIP większość interpretacji zawiera również następujące pola:

1. słowa kluczowe,

2. istotę interpretacji,

3. stanowisko,

4. klasyfikację wg przepisów

Co do tego, jakiego podatku dotyczą - istnieje teoretycznie możliwość klasyfikacji interpretacji podczas jej wprowadzenia i nadania jej odpowiednich słów kluczy czy klasyfikacji wg przepisów. Niemniej doświadczenie mnie nauczyło, że **ten klucz nie zawsze był wprowadzany**. Lepszym miernikiem jest akapit rozpoczynający interpretację; według niego przeprowadziłem klasyfikację. W przypadku problemów, klasyfikacja nastąpiła ręcznie (dotyczy to około 200 interpretacji).

#### Podział na tokeny

Poniżej prosty tokenizer oparty o silnik Morfeusza. Z tego względu, że cały projekt powstawał w Python 3, a biblioteka dynamiczna Morfeusza została napisana na potrzeby Python2 musiałem stworzyć drobne API odpalające analyser z Morfeusza w Python 3.

In [2]:
import execnet

def token_converter(text):
    gw = execnet.makegateway("popen//python=python2.7")

    channel = gw.remote_exec(
        """import morfeusz2 as morf
import gc

def converter(text):
    
    analyzer = morf.Morfeusz()

    text = analyzer.analyse(text)
    
    
            
    return text

text = channel.receive()
channel.send(converter(text))

del converter
del text
del morf
""")
    channel.reconfigure(py2str_as_py3str=False)
    channel.send(text)
    text = channel.receive()
    channel.close()
    execnet.default_group.terminate()
    return text